In [5]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.optimize import curve_fit
from pandas import DataFrame as df
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import get_decimal_separator
from Data_setup_AEMEL_NewCell import data_set
import os
from galvani import BioLogic
from math import floor,log10

fig_title = 'Polarization Curve Tafel steps plus stefano control'
fig = go.Figure()
fig.update_xaxes(range=[0, 1050],
    nticks=6,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[1.2, 2.3],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
colors = px.colors.qualitative.Dark24

fig.update_layout(
    xaxis_title="Current Density (mA/cm<sup>2</sup>)",
    yaxis_title="Cell Voltage (V)",
    margin={'t':0,'l':0,'b':0,'r':10},
    legend_font_size=22,
    font=dict(
        family="Times New Roman, monospace",
        size=28,
        color="black",
    ),
)
fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(255, 255, 255, 255)', #sets the background to be transparent... might appear black in .jpeg but will be transparent in the .svg
})

for i in data_set:
    
    file = data_set[i]['data']
    if not os.path.exists(file):
        print("File not found: " + file)
        continue
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        data = pd.read_csv(file, encoding='ansi', sep="\t", decimal=get_decimal_separator(file),  skiprows=range(headerlength))
        current_column = '<I>/mA'
    else:
        mpr_file = BioLogic.MPRfile(file)
        data = pd.DataFrame(mpr_file.data)
        current_column = 'I/mA'
    if not os.path.exists(data_set[i]['data']):
        continue
   #define variables, sets up number of experiments, does the voltage correction based on Ru and 
    exp = data['Ns']
    Nexp = exp[-1:] + 1
    voltage = data['Ewe/V']
    current = data['I/mA']/5

    j = 0
    Polarization = []
    for j in range(int(Nexp)):
        v = df.mean(voltage.loc[exp==j])
        J = df.mean(current.loc[exp==j])
        verr = df.std(voltage.loc[exp==j])
        Polarization.append([v, J, verr])   
    Polarization = np.array(Polarization)
    
    # fig.add_trace(go.Scatter(
    #     y=Polarization[:,0],
    #     x=Polarization[:,1],
    #     error_y=dict(
    #         type='data',
            
    #         array=Polarization[:,2],
    #         visible=True),
    #     mode='markers',
    #     name= data_set[i]['label'],
    #      showlegend=False,
    #     line_color = colors[data_set[i]['color_index']],
        
    # )
    # )
    # add a trace to the plot with or without a label in the legend
    if data_set[i]['label'] != '':
        fig.add_trace(go.Scatter(
            y=Polarization[:,0],
            x=Polarization[:,1],
            error_y=dict(
                type='data',
                array=Polarization[:,2],
                visible=True),
            mode='markers',
            marker=dict(
                size=6
            ),
            name= data_set[i]['label'],
            line_color = colors[data_set[i]['color_index']],
        )
        )
    else:
        fig.add_trace(go.Scatter(
            y=Polarization[:,0],
            x=Polarization[:,1],
            error_y=dict(
                type='data',
                array=Polarization[:,2],
                visible=True),
            mode='markers',
            marker=dict(
                size=6
            ),
            showlegend=False, # hide the label in the legend if it's empty
            line_color = colors[data_set[i]['color_index']],
        )
        )




   
fig.update_layout(
    width = 500,
    height = 500,
    legend=dict(
    yanchor="top",
    y=1.08,
    xanchor="left",
    x=0,
    font=dict(
        family="Times New Roman, monospace",
        size=28,
        color="black",
    ),
    ))

fig.show()
fig.write_image(fig_title +'.jpeg')
fig.write_image(fig_title + '.svg')
